<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-I:-Preprocessing-and-EDA" data-toc-modified-id="Part-I:-Preprocessing-and-EDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part I: Preprocessing and EDA</a></span><ul class="toc-item"><li><span><a href="#Analyze-Missingness" data-toc-modified-id="Analyze-Missingness-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Analyze Missingness</a></span></li><li><span><a href="#Functions-for-EDA" data-toc-modified-id="Functions-for-EDA-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Functions for EDA</a></span></li><li><span><a href="#Problem-1:-Dataset-Import-&amp;-Cleaning" data-toc-modified-id="Problem-1:-Dataset-Import-&amp;-Cleaning-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Problem 1: Dataset Import &amp; Cleaning</a></span></li><li><span><a href="#Problem-2:-Inventory-Management" data-toc-modified-id="Problem-2:-Inventory-Management-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Problem 2: Inventory Management</a></span></li><li><span><a href="#Problem-3:-Why-did-customers-make-returns?" data-toc-modified-id="Problem-3:-Why-did-customers-make-returns?-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Problem 3: Why did customers make returns?</a></span></li></ul></li></ul></div>

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import time
import random
import statsmodels.api as sm
import sklearn.model_selection as ms
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
sns.set(style = "whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn import neighbors
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor, export_graphviz

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Part I: Preprocessing and EDA
- The data comes from a global e-retailer company, including orders from 2012 to 2015. Import the **Orders** dataset and do some basic EDA. 
- For problem 1 to 3, we mainly focus on data cleaning and data visualizations. You can use all the packages that you are familiar with to conduct some plots and also provide **brief interpretations** about your findings.

- How will we handle missing values?
- Coerce dates to datetime
- USD to float64
- Outliers and How to deal with?
- Does any of the features look skewed? Should we transform them using Box-Cox, log, sqrt?
- Which variables should we dummify?
- Quick correlation plots. Variables with high corrleation to prices should be prioritized as predictors
- How to deal with multi-collinearity?

In [4]:
# import Orders.csv
orders = pd.read_csv('./Data/Orders.csv')

In [13]:
orders

,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.98,2,0.0,62.15,40.770,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.40,9,0.1,-288.77,923.630,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.17,9,0.1,919.97,915.490,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,Germany,Western Europe,Europe,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.160,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,Senegal,Western Africa,Africa,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52,903.040,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2015-KE1642066-42174,2015-06-19,2015-06-19,Same Day,KE-1642066,Katrina Edelman,Corporate,NaN,Kure,Hiroshima,Japan,Eastern Asia,Asia Pacific,OFF-FA-3072,Office Supplies,Fasteners,"Advantus Thumb Tacks, 12 Pack",65.10,5,0.0,4.50,1.010,Medium
51286,34337,US-2014-ZD21925140-41765,2014-05-06,2014-05-10,Standard Class,ZD-219251408,Zuschuss Donatelli,Consumer,37421.0,Chattanooga,Tennessee,United States,Southern US,USCA,FUR-FU-4070,Furniture,Furnishings,"Eldon Image Series Desk Accessories, Burgundy",16.72,5,0.2,3.34,1.930,High
51287,31315,CA-2012-ZD21925140-41147,2012-08-26,2012-08-31,Second Class,ZD-219251404,Zuschuss Donatelli,Consumer,94109.0,San Francisco,California,United States,Western US,USCA,OFF-AR-5321,Office Supplies,Art,Newell 341,8.56,2,0.0,2.48,1.580,High
51288,9596,MX-2013-RB1979518-41322,2013-02-17,2013-02-21,Standard Class,RB-1979518,Ross Baird,Home Office,NaN,Valinhos,São Paulo,Brazil,South America,LATAM,OFF-BI-2919,Office Supplies,Binders,"Acco Index Tab, Economy",13.44,2,0.0,2.40,1.003,Medium


In [14]:
orders.dtypes

Row.ID                     int64
Order.ID                  object
Order.Date        datetime64[ns]
Ship.Date         datetime64[ns]
Ship.Mode                 object
Customer.ID               object
Customer.Name             object
Segment                   object
Postal.Code              float64
City                      object
State                     object
Country                   object
Region                    object
Market                    object
Product.ID                object
Category                  object
Sub.Category              object
Product.Name              object
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
Shipping.Cost            float64
Order.Priority            object
dtype: object

In [ ]:
# USD columns
usd = ['Sales', 'Profit']

# date columns
date_ = ['Order.Date', 'Ship.Date']

In [6]:
orders['Sales'] = orders['Sales'].str.replace('$', '').str.replace(',', '')
orders['Profit'] = orders['Profit'].str.replace('$', '').str.replace(',', '')

In [7]:
orders['Sales'] = orders['Sales'].astype('float64', copy=False)
orders['Profit'] = orders['Profit'].astype('float64', copy=False)

In [11]:
orders['Order.Date']= pd.to_datetime(orders['Order.Date'])
orders['Ship.Date']= pd.to_datetime(orders['Ship.Date'])

In [12]:
orders.dtypes

Row.ID                     int64
Order.ID                  object
Order.Date        datetime64[ns]
Ship.Date         datetime64[ns]
Ship.Mode                 object
Customer.ID               object
Customer.Name             object
Segment                   object
Postal.Code              float64
City                      object
State                     object
Country                   object
Region                    object
Market                    object
Product.ID                object
Category                  object
Sub.Category              object
Product.Name              object
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
Shipping.Cost            float64
Order.Priority            object
dtype: object

### Analyze Missingness

In [6]:
# Identify missing rows and columns
missingRows = pd.isnull(orders).sum(axis=1) > 0
missingCols = pd.isnull(orders).sum(axis=0) > 0
print('The columns with missingness are: %s' % (orders.columns[missingCols]))
print('Number of rows with missingness are: %s' % (np.sum(missingRows)))

The columns with missingness are: Index(['Postal.Code'], dtype='object')
Number of rows with missingness are: 41296


In [7]:
# for columns with missing values, what percent missing?
# Percentage of missing values in columns that are missing values:
def missingValPercent(col, df):
    val = df[col].isnull().sum()/df.shape[0]*100
    return round(val, 1)

missing_cols = pd.concat([pd.Series(orders.columns[missingCols]), 
                          pd.Series([missingValPercent(ele, df = orders) 
                                     for ele in orders.columns[missingCols]])], axis=1, keys=['column', 'percent_missing'])
pd.set_option('display.max_rows', None)
missing_cols

,column,percent_missing
0,Postal.Code,80.5


### Functions for EDA

In [ ]:
def categorical(df, col_name):
    """
    Arguments:
    - df: dataframe
    - col_name: name of column 
    Returns: 
    - count of missing values
    - frequency of each unique value
    - 
    """
    print("Missing Values Count: ", df[col_name].isna().sum())
    print("\nUnique values and frequency:")
    print(df[col_name].value_counts(sort = True))

In [ ]:
def continuous(df, col_name):
    """
    Arguments:
    - df: dataframe
    - col_name: name of column 
    Returns:
    - count of missing values
    - statistical summary
    - distribution
    """
    print("Missing Values Count: ", df[col_name].isna().sum())
    print("\nStatistical summary:")
    print(df[col_name].describe())
    df[col_name].plot.hist()

In [11]:
cat = orders.select_dtypes(["object","category"])
cont = orders.select_dtypes(["float64","int64"]) 

In [13]:
cat.head(5)

,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Profit,Order.Priority
0,CA-2014-AB10015140-41954,11/11/14,11/13/14,First Class,AB-100151402,Aaron Bergman,Consumer,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,$221.98,$62.15,High
1,IN-2014-JR162107-41675,2/5/14,2/7/14,Second Class,JR-162107,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black","$3,709.40",-$288.77,Critical
2,IN-2014-CR127307-41929,10/17/14,10/18/14,First Class,CR-127307,Craig Reiter,Consumer,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID","$5,175.17",$919.97,Medium
3,ES-2014-KM1637548-41667,1/28/14,1/30/14,First Class,KM-1637548,Katherine Murray,Home Office,Berlin,Berlin,Germany,Western Europe,Europe,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless","$2,892.51",-$96.54,Medium
4,SG-2014-RH9495111-41948,11/5/14,11/6/14,Same Day,RH-9495111,Rick Hansen,Consumer,Dakar,Dakar,Senegal,Western Africa,Africa,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed","$2,832.96",$311.52,Critical


In [14]:
cont.head(5)

,Row.ID,Postal.Code,Quantity,Discount,Shipping.Cost
0,40098,73120.0,2,0.0,40.77
1,26341,NaN,9,0.1,923.63
2,25330,NaN,9,0.1,915.49
3,13524,NaN,5,0.1,910.16
4,47221,NaN,8,0.0,903.04


In [ ]:
cat.apply(categorical)

### Problem 1: Dataset Import & Cleaning
Check **"Profit"** and **"Sales"** in the dataset, convert these two columns to numeric type. 

### Problem 2: Inventory Management
- Retailers that depend on seasonal shoppers have a particularly challenging job when it comes to inventory management. Your manager is making plans for next year's inventory.
- He wants you to answer the following questions:
    1. Is there any seasonal trend of inventory in the company?
    2. Is the seasonal trend the same for different categories?

- ***Hint:*** For each order, it has an attribute called `Quantity` that indicates the number of product in the order. If an order contains more than one product, there will be multiple observations of the same order.

### Problem 3: Why did customers make returns?
- Your manager required you to give a brief report (**Plots + Interpretations**) on returned orders.

	1. How much profit did we lose due to returns each year?

	2. How many customer returned more than once? more than 5 times?

	3. Which regions are more likely to return orders?

	4. Which categories (sub-categories) of products are more likely to be returned?

- ***Hint:*** Merge the **Returns** dataframe with the **Orders** dataframe using `Order.ID`.